In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense ,Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import visualkeras
import time

In [ ]:
train_data_dir=r'C:\Users\a\OneDrive\Desktop\ABEER\proj2\classifier\training dataset'
test_data_dir=r'C:\Users\a\OneDrive\Desktop\ABEER\proj2\classifier\testing dataset'

In [ ]:
img_width, img_height = 64,64
batch_size = 64
validation_split = 0.2

In [ ]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,  # Normalize pixel values to the range [0, 1]
    rotation_range=20,   # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,  # Apply shear transformation
    zoom_range=0.2,  # Randomly zoom images
    horizontal_flip=True,  # Randomly flip images horizontally
    validation_split=validation_split
)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

In [ ]:
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width,img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
model = tf.keras.Sequential([
    Conv2D(64, (5, 5), activation='relu', strides=(1,1), input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(128, (5, 5), activation='relu', strides=(1, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(256, (5, 5), activation='relu', strides=(1, 1)),
    Conv2D(256, (5, 5), activation='relu', strides=(1, 1)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.3)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
epochs = 20
history=model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Test accuracy: {accuracy * 100:.2f}% \n')


In [ ]:
y_pred_probs = model.predict(test_generator, steps=len(test_generator))
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = predict_generator.classes
conf_matrix = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels = train_generator.class_indices.keys(), yticklabels = predict_generator.class_indices.keys())

plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
class_names = list(predict_generator.class_indices.keys())
report = classification_report(y_true, y_pred, target_names=class_names)

# Display the classification report
print("Classification Report:")
print(report)


In [ ]:
model.save("kidney.h5")

In [ ]:
model=keras.models.load_model("kidney.h5")

In [ ]:
import keras